In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

# FAISS를 사용한 KMeans 클러스터링 클래스
class FaissKMeans:
    def __init__(self, n_clusters=8, n_init=10, max_iter=300):
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.kmeans = None
        self.cluster_centers_ = None
        self.inertia_ = None

    def fit(self, X):
        self.kmeans = faiss.Kmeans(d=X.shape[1], k=self.n_clusters,
                                   niter=self.max_iter, nredo=self.n_init)
        self.kmeans.train(X.astype(np.float32))
        self.cluster_centers_ = self.kmeans.centroids
        self.inertia_ = self.kmeans.obj[-1]

    def predict(self, X):
        _, I = self.kmeans.index.search(X.astype(np.float32), 1)
        return I.squeeze()

# 문장 임베딩 모델 로드
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2') # 이거 좋음 ㅇㅇ

# 데이터셋 불러오기
dataset = load_dataset("maywell/synatra_solar_ko", split="train")

# 'text' 칼럼의 문장들을 임베딩
embeddings = model.encode(dataset['output'], show_progress_bar=True)

In [ ]:
faiss_kmeans = FaissKMeans(n_clusters=6, n_init=50, max_iter=600)
faiss_kmeans.fit(embeddings)
cluster_assignments = faiss_kmeans.predict(embeddings)

In [ ]:
k = 0
# 결과 출력 (예시: 각 클러스터에 속하는 첫 번째 문장 출력)
for i in range(6):
    cluster_indices = [index for index, cluster_id in enumerate(cluster_assignments) if cluster_id == i]
    print(f"Cluster {i+1}:")
    print(len(cluster_indices))
    k += len(cluster_indices)
    for j in cluster_indices[:1]:  # 각 클러스터에서 첫 번째 문장만 출력
        print(dataset['output'][j])
    print("\n")
    
print(f"총 {k}개")

In [ ]:
from datasets import Dataset
from tqdm.auto import tqdm

def create_cluster_data(example, index, cluster_assignments):
    # 해당 데이터가 속한 클러스터를 확인하고 클러스터 ID를 반환
    cluster_id = cluster_assignments[index]
    return {'cluster_id': cluster_id}

# 각 데이터에 대한 클러스터 ID 할당
with tqdm(total=len(dataset), desc="Processing clusters") as pbar:
    cluster_dataset = dataset.map(create_cluster_data, with_indices=True, num_proc=4, 
                                  batched=False, fn_kwargs={"cluster_assignments": cluster_assignments},
                                  )


In [ ]:
# 각 클러스터 별로 데이터셋 분리 및 업로드
for i in range(faiss_kmeans.n_clusters):
    # 해당 클러스터에 속하는 데이터 필터링
    cluster_data = cluster_dataset.filter(lambda x: x['cluster_id'] == i)

    # 필요한 컬럼만 선택
    cluster_data = cluster_data.map(lambda x: {'instruction': x['instruction'], 'output': x['output']})

    # Hugging Face Hub에 업로드
    cluster_data.push_to_hub(f"maywell/cluster_{i}")

In [ ]:
# Mergekit config
import yaml
import random

# YAML 파일에 들어갈 기본 구조 설정
yaml_data = {
    'base_model': './out-merge-2',
    'gate_mode': 'hidden',
    'dtype': 'bfloat16',
    'experts': []
}

# 각 클러스터에 대한 데이터 추가
for i in range(faiss_kmeans.n_clusters):
    # 해당 클러스터 데이터셋 필터링
    cluster_data = cluster_dataset.filter(lambda x: x['cluster_id'] == i)

    # 무작위로 100개의 'output' 샘플 선택
    positive_prompts = random.sample(cluster_data['output'], min(100, len(cluster_data['output'])))

    # 클러스터 별 전문가 모델 정보 추가
    yaml_data['experts'].append({
        'source_model': f'./cluster-{i}',
        'positive_prompts': positive_prompts
    })

# YAML 파일로 저장
with open('config.yml', 'w') as file:
    yaml.dump(yaml_data, file, sort_keys=False, allow_unicode=True)
